You can use RDDs when you want to perform low-level transformations and actions on your unstructured data. 

RDDs
RDDs are the building blocks of Spark. It’s the original API that Spark exposed and pretty much all the higher level APIs decompose to RDDs. From a developer’s perspective, an RDD is simply a set of Java or Scala objects representing data.

RDDs have three main characteristics: they are compile-time type safe (they have a type!), they are lazy and they are based on the Scala collections API.

RDD is a distributed collection of data elements without any schema-- can have unstructured data tooo

In [10]:


import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [6]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 4234-2518

 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK

05-04-2021  09:47 AM    <DIR>          .
05-04-2021  09:47 AM    <DIR>          ..
05-04-2021  09:45 AM    <DIR>          .ipynb_checkpoints
03-04-2021  12:18 PM           292,486 1.Context , session and data import.ipynb
03-04-2021  07:35 PM            99,155 2. Schemas_Columns_rows_groupby_sort_unique.ipynb
03-04-2021  08:37 PM            23,185 3. Built_in_functions.ipynb
05-04-2021  09:43 AM            84,261 3.Functions&date&joins.ipynb
03-04-2021  09:23 AM             4,046 Download_data.ipynb
03-04-2021  07:33 PM             2,294 FIRST_SAMPLE.ipynb
19-08-2019  06:56 AM             5,699 police_station.csv
05-04-2021  09:47 AM               555 RDD.ipynb
02-04-2021  04:48 PM       171,057,223 reported-crimes.csv
02-04-2021  04:48 PM       171,057,223 rows.csv@accessType=DOWNLOAD
03-04-2021  07:42 PM             8,933 Untitled.ipynb
03-04-2021  12:

In [7]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [8]:
from pyspark.sql.functions import to_timestamp,col,lit
ps = spark.read.csv('police_station.csv',header=True)
ps.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -

### RDD SETUP

In [12]:
psrdd=sc.textFile("police_station.csv")
psrdd

police_station.csv MapPartitionsRDD[14] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
psrdd.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [14]:
ps_header=psrdd.first()

In [15]:
ps_rest=psrdd.filter(lambda line:line!=ps_header)
ps_rest.first()

'Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,,,,1177731.401,1881697.404,41.83070169,-87.62339535,"(41.8307016873, -87.6233953459)"'

##### How many police stations are there?

In [16]:
ps_rest.map(lambda line:line.split(",")).collect()

[['Headquarters',
  'Headquarters',
  '3510 S Michigan Ave',
  'Chicago',
  'IL',
  '60653',
  'http://home.chicagopolice.org',
  '',
  '',
  '',
  '1177731.401',
  '1881697.404',
  '41.83070169',
  '-87.62339535',
  '"(41.8307016873',
  ' -87.6233953459)"'],
 ['18',
  'Near North',
  '1160 N Larrabee St',
  'Chicago',
  'IL',
  '60610',
  'http://home.chicagopolice.org/community/districts/18th-district-near-north/',
  '312-742-5870',
  '312-742-5771',
  '312-742-5773',
  '1172080.029',
  '1908086.527',
  '41.90324165',
  '-87.64335214',
  '"(41.9032416531',
  ' -87.6433521393)"'],
 ['19',
  'Town Hall',
  '850 W Addison St',
  'Chicago',
  'IL',
  '60613',
  'http://home.chicagopolice.org/community/districts/19th-district-town-hall/',
  '312-744-8320',
  '312-744-4481',
  '312-744-8011',
  '1169730.744',
  '1924160.317',
  '41.94740046',
  '-87.65151202',
  '"(41.9474004564',
  ' -87.651512018)"'],
 ['20',
  'Lincoln',
  '5400 N Lincoln Ave',
  'Chicago',
  'IL',
  '60625',
  'http://

In [17]:
ps_rest.map(lambda line:line.split(",")).count()

24

### display district id,name,a ddres and zip for distric id 7

In [23]:
(ps_rest.filter(lambda line:line.split(",")[0]=="7").map(lambda line:(line.split(",")[0],
                                                     line.split(",")[0],
                                                     line.split(",")[1],
                                                     line.split(",")[5])).collect())

[('7', '7', 'Englewood', '60636')]